# Optimized Meal Planning

## Define the model

Data

$$
\begin{aligned}
T, \text{the set of discrete time periods} \\
I, \text{the set of ingredients that are available} \\
N, \text{the set of nutrients that are tracked} \\
R, \text{the set of known recipes} \\
F_{in}, \text{the amount of nutrient } n \text{ in ingredient } i,\ \forall i \in I,\ n \in N \\
S_{n}, \text{the amount of nutrient } n \text{ that is needed each week for a healthy diet} \\
C_{i}, \text{the cost of a single quantity of ingredient } i,\ \forall i \in I \\
V_{i}, \text{the volume of a single quantity of ingredient } i,\ \forall i \in I \\
Fr, \text{the max volume of the fridge} \\
Req_{ir}, \text{the amount of ingredient i needed for recipe} \ l \ \forall i \in I, r \in R \\
MAX, \text{the maximum number of recipes that can be made in a week}
\end{aligned}
$$

Only consider whether or not to buy an ingredient as an integer variable. The argument for this is that in real life, a recipe can be made with any amount of an ingredient but stores usually sell ingredients in preset sizes.

Decision variables:
- $x_{it}$, binary variable representing if ingredient i bought in week k, $\forall i \in I, t \in T$
- $y_{rt}$, the amount of recipe $l$ to make in week k, $\forall r \in R, t \in T$
- $z_{it}$, the amount of ingredient i to store in the fridge in week k, $\forall i \in I, t \in T$

Integer Program:
\begin{align}
\min \sum_{t \in T} \sum_{i \in I} x_{it} * C_i \\
\text{s.t. } \sum_{r \in R} \sum_{i \in I} y_{rt} * Req_{ir} * F_{in} \geq S_{n}, \forall t \in T, n \in N \\
\sum_{i \in I} z_{it} * V_{i} \leq Fr, \forall t \in T \\
z_{i0} = 0, \forall i \in I \\
x_{it}*V_i + z_{i,t-1} = \sum_{r \in R} y_{rt} * Req_{ir} + z_{it}, \forall t \in T, i \in I \\
\sum_{r \in R} y_{rt} \leq MAX, \forall t \in T \\
y_{rt}, z_{it} \geq 0, \forall i \in I, t \in T, x_{it} \text{ is binary}
\end{align}

- The first constraint ensures there are enough nutrients
- The second constraint ensures that the fridge is not overfilled
- The third constraint sets the intial amount of food to 0
- The fourth constraint ensures that the amount of each ingredient being consumed is balanced by the amount being bought
- The fifth constraint limits the amount of recipes made in a week (i.e. only 7 recipes worth of recipes being made)

In [ ]:
using Random, NamedArrays, Plots

T_end = 10
T = 1:T_end
FR_CAP = 100

#Data 2:
Random.seed!(300)

# INGREDIENT DATA
I = [:rice, :chicken, :steak, :noodles, :cream, :tomato, :asparagus, :bread, :tofu, :banana, :yogurt, :cereal]

# C init is now the cost of a single pack of the ingredient - from metromarket.com
Cinit = [2, 13, 19, 1.3, 10, 3.8, 6.5, 1.9, 2, 1.6, 1.5, 6.3] # dollars

# the cost is accompanied by a volume corresponding to the size of that package
V = NamedArray([32, 80, 10, 16, 32, 10, 12, 20, 16, 4, 4.4, 15.4], # oz
                I, "Ingredients")
 
N = [:calories, :protein, :fiber, :carbs, :fat, :vitamins, :minerals]

R = [:steak_rice_asparagus, :chicken_ceasar_salad, :chicken_marinara, 
    :steak_alfredo, :banana_toast, :overnight_oats, :stirfried_tofu, :clamless_chowder, :CLT]

# g / oz
F = NamedArray([37 .76 .11 8 .08 1 1;
        47 8.8 0 0 1 .25 3;
        53 8.3 0 0 2.2 .25 3.5;
        45 1.6 .51 8.7 .26 0 .5;
        96 .81 0 .81 10.2 1 1;
        5 .25 .34 1.1 .06 3 1.5;
        6 .62 .6 1.1 .03 2 2;
        76 2.5 .77 14 .95 0 1.5;
        41 4.9 .65 .79 2.5 1 3;
        25 .31 .74 6.5 .09 1.5 3.5;
        17 .98 0 1.3 .92 .5 1.25;
        92 2.2 3.9 22.4 .45 .75 3;
        ], (I, N), ("Ingredients", "Nutrients"))

# INGREDIENT DATA

# Each recipe uses about 10 total units of ingredient
Req = NamedArray([  5 0 3 0 0 0 3 0 0 0 0 0; # steak rice aspargus
                    0 3 0 0 0 2 2 0 0 0 1 0; # chicken ceasar salad
                    0 2 0 5 0 3 0 0 0 0 0 0; # chicken marinara
                    0 0 3 5 2 0 0 0 0 0 0 0; # streak alfredo
                    0 0 0 0 0 0 0 1 0 1 0 0; # banana toast
                    0 0 0 0 0 0 0 0 0 1 1 3; # overnight oats
                    1 0 0 0 0 0 1 0 1 0 0 0; # stirfried tofu
                    0 1 0 0 3 0 0 1 0 0 0 0; # clamless chowder
                    0 1 0 0 0 1 0 1 0 0 0 0; # chicken lettuce tomato
                    ],
                    (R, I), ("Recipes", "Ingredients")) 

# nutrition requirements according to Google for average sized adult male
# calories: 2000 kcals
# protein: 55 g (bhf.org.uk)
# fiber: 40 g (health.harvard.edu)
# carbs: 130 g (mayoclinic.org)
# fat:50
# vitamins: 10 (total amount of vitamins needed)
# minerals: 10 (total amount of minerals needed)
S = NamedArray([2000, 55, 40, 130, 50, 10, 10], N, "Nutrient Requirements")

C = NamedArray(zeros(Float64, length(T), length(I)), (T, I), ("Week", "Ingredients"))

# Creating prices that randomly over time
for i in 1:length(I)
    C[1, i] = length(String(I[i]))
end

for t in 2:length(T)
    for i in 1:length(I)
        C[t, i] = C[t - 1, i] + .5 * Random.randn() + .025 * C[t - 1, i]
        if C[t, i] <= 1
            C[t, i] = 1.0
        end
    end
end


# make the model
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, x[T, I], Bin)
@variable(m, y[T, R] >= 0)
@variable(m, z[T, I] >= 0)

# Indexing Guide:
#
# t - time, i - ingredient, r - recipe, n - nutrient
# x[t, i], y[t, r], Req[r, i], F[i, n]

@constraint(m, nutr_satisfied[n in N, t in T],
    sum(y[t, r] * sum(Req[r, i] * F[i, n] for i in I) for r in R) >= S[n])

#@constraint(m, enough_ingr_for_recipes[i in I, t in T],
  #  x[t, i]*V[i] >= sum( Req[r, i]*y[t, r] for r in R )) 

@constraint(m, fridge_capacity[t in T], sum( x[t, i]*V[i] for i in I ) <= FR_CAP)

@constraint(m, first_ingredient_bal[i in I], x[1, i]*V[i] == sum( y[1,r] * Req[r,i] for r in R) + z[1, i])

@constraint(m, ingredient_bal[t in 2:T_end, i in I], x[t, i]*V[i] + z[t-1, i] == sum( y[t,r] * Req[r,i] for r in R) + z[t, i])

@objective(m, Min, sum( x[t, i] * C[t, i] for i in I, t in T ))

set_silent(m)
optimize!(m)

ingredient_values = NamedArray( [ (value(x[t,i])) for t in T, i in I ], (T, I), ("Week", "Ingredients"))
recipe_values = NamedArray( [ (value(y[t,r])) for t in T, r in R ], (T, R), ("Week", "Recipes"))
fridge_values = NamedArray( [ (value(z[t,i])) for t in T, i in I ], (T, I), ("Week", "Stored"))

println("Cost Matrix")
println(C)
println()

p1 = plot(1:10, C, xlabel="Week", ylabel="Cost" , label=permutedims(String.(I)))
display(p1)
println()

println("What ingredients to buy in each week")
display(ingredient_values)
println()

p2 = plot(1:10, ingredient_values, xlabel="Week", ylabel="Ingredient bought?" , label=permutedims(String.(I)))
display(p2)
println()

println("Recipes cooked per week")
display(recipe_values)
println()

p3 = plot(1:10, recipe_values, xlabel="Week", ylabel="Quantity of Recipe Cooked" , label=permutedims(String.(R)))
display(p3)
println()

println("How much of each ingredient to store each week")
display(fridge_values)
println()

p4 = plot(1:10, fridge_values, xlabel="Week", ylabel="Quantity Stored" , label=permutedims(String.(I)))
display(p4)
println()

println("Total cost ", objective_value(m))

In [ ]:
#using DataFrames, CSV
#df = CSV.read("stigler.csv",DataFrame,delim=',')